# Entrega 2 - Proyecto 1
### Nelson Escalante - 22046

### SETUP

In [2]:
#importar pandas
import pandas as pd
import matplotlib.pyplot as plt

#convertir los datos de movies.csv en un dataframe para trabajar
path = "../movies.csv"
moviesdf = pd.read_csv(path, encoding='latin_1')
moviesdf

,id,budget,genres,homePage,productionCompany,productionCompanyCountry,productionCountry,revenue,runtime,video,...,popularity,releaseDate,voteAvg,voteCount,genresAmount,productionCoAmount,productionCountriesAmount,actorsAmount,castWomenAmount,castMenAmount
0,5,4000000,Crime|Comedy,https://www.miramax.com/movie/four-rooms/,Miramax|A Band Apart,US|US,United States of America,4257354.0,98,False,...,20.880,1995-12-09,5.7,2077,2,2,1,25,15,9
1,6,21000000,Action|Thriller|Crime,NaN,Universal Pictures|Largo Entertainment|JVC,US|US|JP,Japan|United States of America,12136938.0,110,False,...,9.596,1993-10-15,6.5,223,3,3,2,15,3,9
2,11,11000000,Adventure|Action|Science Fiction,http://www.starwars.com/films/star-wars-episod...,Lucasfilm|20th Century Fox,US|US,United States of America,775398007.0,121,NaN,...,100.003,1977-05-25,8.2,16598,3,2,1,105,5,62
3,12,94000000,Animation|Family,http://movies.disney.com/finding-nemo,Pixar,US,United States of America,940335536.0,100,NaN,...,134.435,2003-05-30,7.8,15928,2,1,1,24,5,18
4,13,55000000,Comedy|Drama|Romance,NaN,Paramount|The Steve Tisch Company,US|,United States of America,677387716.0,142,False,...,58.751,1994-07-06,8.5,22045,3,2,1,76,18,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,920081,0,Action|Horror,NaN,NaN,NaN,NaN,0.0,100,False,...,16.662,2021-11-26,6.8,108,2,1,1,10,2,4
9996,920143,0,Comedy,NaN,Caracol Televisión|Dago García Producciones,CO|CO,Colombia,0.0,97,False,...,491.706,2021-12-25,1.5,2,1,2,1,8,1,1
9997,922017,0,Comedy,NaN,NaN,NaN,Nigeria,0.0,112,False,...,565.658,2021-12-17,6.1,30,1,1,17,1,0,922017
9998,922162,0,NaN,https://www.netflix.com/title/81425229,NaN,NaN,United States of America,0.0,59,False,...,9.664,2021-12-17,6.0,1,1,0,0,0,922162,The Witcher: Fireplace


## Seccion 1 - Clustering

### Ejercicio 1: Haga el preprocesamiento del dataset, explique que variables no aportan informacion a la generacion de grupos y por que.

In [3]:
#Mostrar las columnas para evaluar las variables
moviesdf.columns

Index(['id', 'budget', 'genres', 'homePage', 'productionCompany',
       'productionCompanyCountry', 'productionCountry', 'revenue', 'runtime',
       'video', 'director', 'actors', 'actorsPopularity', 'actorsCharacter',
       'originalTitle', 'title', 'originalLanguage', 'popularity',
       'releaseDate', 'voteAvg', 'voteCount', 'genresAmount',
       'productionCoAmount', 'productionCountriesAmount', 'actorsAmount',
       'castWomenAmount', 'castMenAmount'],
      dtype='object')

Las variables que no aportan valor son:
- id: Solo sirve para trabajar con los datos, no tiene relacion con las peliculas.
- homePage: Es solo la pagina de una pelicula, no influye con su popularidad.
- video: Solamente nos dice si cuenta con un video promocional o no.
- actorsCharacter: Es informacion que solo sirve en la trama de la pelicula, y no se traslada al mundo real.
- originalTitle y title: No podemos realizar analisis en base a los titulos de las peliculas.

In [5]:
from sklearn.preprocessing import StandardScaler

#Enlistar las columnas que no aportan valor para el clustering
useless = ['id', 'homePage', 'video', 'actorsCharacter', 'originalTitle', 'title']

#Enlistar las variables numericas
quant = ['budget', 'revenue', 'runtime', 'popularity', 'voteAvg', 'actorsAmount']

#Eliminar dichas columnas
proc_movies = moviesdf.drop(columns=useless)
proc_movies = proc_movies.dropna()

#Normalizar las variables cuantitativas
scaler = StandardScaler()
proc_movies[quant] = scaler.fit_transform(proc_movies[quant])

proc_movies

,budget,genres,productionCompany,productionCompanyCountry,productionCountry,revenue,runtime,director,actors,actorsPopularity,...,popularity,releaseDate,voteAvg,voteCount,genresAmount,productionCoAmount,productionCountriesAmount,actorsAmount,castWomenAmount,castMenAmount
0,-0.445547,Crime|Comedy,Miramax|A Band Apart,US|US,United States of America,-0.382643,-0.186524,Allison Anders|Alexandre Rockwell|Robert Rodri...,Tim Roth|Jennifer Beals|Antonio Banderas|Valer...,22.225|23.519|17.816|19.893|9.027|7.147|7.769|...,...,-0.142447,1995-12-09,-0.928525,2077,2,2,1,-0.207641,15,9
1,-0.005541,Action|Thriller|Crime,Universal Pictures|Largo Entertainment|JVC,US|US|JP,Japan|United States of America,-0.333098,0.301243,Stephen Hopkins,Emilio Estevez|Cuba Gooding Jr.|Denis Leary|St...,9.008|6.383|10.757|18.295|11.772|14.777|9.669|...,...,-0.192320,1993-10-15,-0.038544,223,3,3,2,-0.646809,3,9
2,-0.264368,Adventure|Action|Science Fiction,Lucasfilm|20th Century Fox,US|US,United States of America,4.466153,0.748364,George Lucas,FALSE,11.881|24.542|14.434|10.651|6.888|5.811|1.432|...,...,0.207259,1977-05-25,1.852665,16598,3,2,1,3.305701,5,62
3,1.883897,Animation|Family,Pixar,US,United States of America,5.503251,-0.105230,Andrew Stanton,FALSE,9.79|8.084|8.538|33.379|11.733|11.866|8.141|12...,...,0.359440,2003-05-30,1.407675,15928,2,1,1,-0.251558,5,18
4,0.874471,Comedy|Drama|Romance,Paramount|The Steve Tisch Company,US|,United States of America,3.849882,1.601958,Robert Zemeckis,Tom Hanks|Robin Wright|Gary Sinise|Mykelti Wil...,33.297|13.246|6.748|6.402|13.189|10.974|10.259...,...,0.024934,1994-07-06,2.186408,22045,3,2,1,2.032114,18,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9988,-0.549078,Comedy,Broke and Bones|Jesse Collins Entertainment|Ir...,GB||US,United Kingdom|United States of America,-0.409413,-1.731123,Josh Ruben|Jack Clough,Hugh Grant|Joe Keery|William Jackson Harper|Lu...,11.75|11.16|11.015|15.229|7.595|1.96|10.748|20...,...,-0.128078,2021-12-27,-0.483534,121,1,3,2,0.495027,14,26
9991,-0.549078,Drama,Malvalanda,ES,Spain,-0.409413,-4.169962,José Manuel Carrasco,Javier Pereira|Marina Salas,0.6|10.738,...,-0.108556,2013-06-13,-5.044686,1,1,1,1,-1.217727,1,0
9992,-0.549078,Comedy|Crime,Elefantec Global|MarVista Entertainment|Partic...,|US|US,United States of America|Mexico,-0.409413,-1.040118,Alonso Iñiguez,Diana Bovio|Ricardo Polanco|Paco Rueda|Hernán ...,2.823|2.183|1.38|3.1|1.22|0.6|0.6|2.683|0.968,...,0.442183,2021-12-11,1.073932,22,2,3,2,-0.910310,4,2
9996,-0.549078,Comedy,Caracol Televisión|Dago García Producciones,CO|CO,Colombia,-0.409413,-0.227172,Rodrigo Triana,Amparo Grisales|Jhon Álex Toro|Chichila Navia|...,1.55|0.84|0.6|0.6|0.6|0.6|0.6|0.6,...,1.938496,2021-12-25,-5.600924,2,1,2,1,-0.954226,1,1


In [ ]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
from random import sample
from numpy.random import uniform

# Función para calcular el estadístico de Hopkins
def hopkins_statistic(X, sample_size=50):
    n, d = X.shape
    sample_size = min(sample_size, n)  # Asegurar que el tamaño de la muestra no sea mayor al dataset
    
    # Normalizar los datos
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Seleccionar puntos aleatorios reales del dataset
    real_samples = X_scaled[sample(range(n), sample_size)]
    
    # Generar puntos aleatorios dentro del mismo rango de valores
    min_vals, max_vals = X_scaled.min(axis=0), X_scaled.max(axis=0)
    random_samples = np.array([uniform(min_vals, max_vals) for _ in range(sample_size)])
    
    # Calcular distancias más cercanas para ambas muestras
    nn = NearestNeighbors(n_neighbors=1).fit(X_scaled)
    real_distances = nn.kneighbors(real_samples, return_distance=True)[0].sum()
    random_distances = nn.kneighbors(random_samples, return_distance=True)[0].sum()
    
    return random_distances / (real_distances + random_distances)


# Seleccionar solo las variables numéricas
proc_num = proc_movies[quant].dropna()

# Calcular el estadístico de Hopkins
hopkins_score = hopkins_statistic(proc_num, sample_size=100)
print(f"Estadístico de Hopkins: {hopkins_score:.4f}")


FileNotFoundError: [Errno 2] No such file or directory: 'ruta/dataset_preprocesado.csv'